In [1]:
# Mean absolute error
def MAE(y, y_pred):
    return np.mean(np.abs(y - y_pred))

In [ ]:
# from LinearRegression
# Root mean squared error
def RMSE(y, y_pred):
    mse = np.mean(np.square(y - y_pred))  # MSE
    return np.sqrt(mse)  # RMSE


In [ ]:
# from LinearRegression
# Verify RSS score
def RSS(y, y_pred):
    return np.sum(np.square(np.subtract(y, y_pred)))

In [ ]:
#Ordinary least square from LinearRegression
def compute_ols_with_noise(temp_C, users):

    # Convert to degrees Fahrenheit
    temp_F = 1.8 * temp_C + 32

    # Add small variations
    noise = np.random.normal(loc=0, scale=0.01, size=temp_F.shape)
    temp_F += noise

    # Create input matrix X
    X = np.c_[temp_C, temp_F]

    # Compute OLS using lstsq
    X1 = np.c_[np.ones(X.shape[0]), X]  # Create X1 matrix
    w, rss, rank, _ = lstsq(X1, users)  # OLS

    return w, rss, rank, X1

In [ ]:
#Ridge  with Regularization from LinearRegression
from sklearn.linear_model import Ridge


def compute_with_regularization(temp_C, users):

    # Add small variations
    noise = np.random.normal(loc=0, scale=0.01, size=temp_C.shape)
    temp_F = (1.8 * temp_C + 32) + noise

    # Create input matrix X
    X = np.c_[temp_C, temp_F]

    # Fit a Ridge regression
    ridge = Ridge(alpha=100)
    ridge.fit(X, users)

    return ridge, X


In [ ]:
#Error surface fron gradientDescent
# Plot the error surface
def visualize_steps(fig, axis, log_a, log_b, x, y):
    # Define a grid of a,b parameters
    min_ab = min(min(log_a), min(log_b))
    max_ab = max(max(log_a), max(log_b))

    d = max_ab - min_ab
    min_ab -= d * 0.1
    max_ab += d * 0.1

    a = np.linspace(min_ab, max_ab, num=40)
    b = np.linspace(min_ab, max_ab, num=40)
    a_grid, b_grid = np.meshgrid(a, b)

    # Compute the RMSE score for each a,b pair on that grid
    rmse_grid = np.zeros_like(a_grid)

    for i in range(40):
        for j in range(40):
            a, b = a_grid[i, j], b_grid[i, j]
            rmse_grid[i, j] = rmse(a * x + b, y)

    # RMSE surface
    axis.set_aspect("equal", adjustable="box")
    mpl_contourset = axis.contourf(a_grid, b_grid, rmse_grid, 20, cmap=plt.cm.coolwarm)
    fig.colorbar(mpl_contourset, ax=axis, label="RMSE")

    # Plot the GD steps
    axis.plot(log_a, log_b, c="#00abe9")
    axis.scatter(log_a, log_b, c="#00abe9")

    # Set titles and labels
    axis.set_xlabel("parameter a")
    axis.set_ylabel("parameter b")

    axis.set_xlim(min_ab, max_ab)
    axis.set_ylim(min_ab, max_ab)